# IMPORT STATEMENTS

In [1]:
from pywebio.input import input, FLOAT
from pywebio.output import put_text
from pywebio.input import *
from pywebio.output import *

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix

# GRABBING DATABASE INFO

In [2]:
def readFile(diag_fn, labs_fn):
    diag_file = open(diag_fn)
    labs_fn = open(labs_fn)
    
    diagnosis_admissions_table = pd.read_table(diag_fn)
    
    labs_table = pd.read_table(labs_fn)
    labs_table['PatientID'] =  labs_table['PatientID'] + '_' + labs_table['AdmissionID'].astype(str)
            
    return diagnosis_admissions_table, labs_table

In [32]:
diag_table, labs_table = readFile('AdmissionsDiagnosesCorePopulatedTable_100.txt', 'LabsCorePopulatedTable_100.txt')

In [93]:
labs_table

,PatientID,AdmissionID,LabName,LabValue,LabUnits,LabDateTime
0,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,1,URINALYSIS: RED BLOOD CELLS,1.8,rbc/hpf,1992-07-01 01:36:17.910
1,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,1,METABOLIC: GLUCOSE,103.3,mg/dL,1992-06-30 09:35:52.383
2,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,1,CBC: MCH,35.8,pg,1992-06-30 03:50:11.777
3,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,1,METABOLIC: CALCIUM,8.9,mg/dL,1992-06-30 12:09:46.107
4,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,1,CBC: RED BLOOD CELL COUNT,4.8,m/cumm,1992-07-01 01:31:08.677
...,...,...,...,...,...,...
111478,F0B53A2C-98CA-415D-B928-E3FD0E52B22A_6,6,CBC: HEMOGLOBIN,13.6,gm/dl,2012-04-21 03:23:50.883
111479,F0B53A2C-98CA-415D-B928-E3FD0E52B22A_6,6,CBC: MCH,26.2,pg,2012-04-21 03:18:44.127
111480,F0B53A2C-98CA-415D-B928-E3FD0E52B22A_6,6,METABOLIC: CHLORIDE,104.4,mmol/L,2012-04-21 02:58:57.840
111481,F0B53A2C-98CA-415D-B928-E3FD0E52B22A_6,6,METABOLIC: GLUCOSE,63.3,mg/dL,2012-04-21 02:50:04.503


In [111]:
labs_table = labs_table.drop('AdmissionID', axis=1)

In [112]:
labs_table

,PatientID,LabName,LabValue,LabUnits,LabDateTime
0,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,URINALYSIS: RED BLOOD CELLS,1.8,rbc/hpf,1992-07-01 01:36:17.910
1,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,METABOLIC: GLUCOSE,103.3,mg/dL,1992-06-30 09:35:52.383
2,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,CBC: MCH,35.8,pg,1992-06-30 03:50:11.777
3,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,METABOLIC: CALCIUM,8.9,mg/dL,1992-06-30 12:09:46.107
4,1A8791E3-A61C-455A-8DEE-763EB90C9B2C_1,CBC: RED BLOOD CELL COUNT,4.8,m/cumm,1992-07-01 01:31:08.677
...,...,...,...,...,...
111478,F0B53A2C-98CA-415D-B928-E3FD0E52B22A_6,CBC: HEMOGLOBIN,13.6,gm/dl,2012-04-21 03:23:50.883
111479,F0B53A2C-98CA-415D-B928-E3FD0E52B22A_6,CBC: MCH,26.2,pg,2012-04-21 03:18:44.127
111480,F0B53A2C-98CA-415D-B928-E3FD0E52B22A_6,METABOLIC: CHLORIDE,104.4,mmol/L,2012-04-21 02:58:57.840
111481,F0B53A2C-98CA-415D-B928-E3FD0E52B22A_6,METABOLIC: GLUCOSE,63.3,mg/dL,2012-04-21 02:50:04.503


In [158]:
# labs_table['PatientID'].tolist()

In [145]:
grouped_labs = labs_table.groupby(['PatientID', 'LabName']).aggregate(np.mean)
# grouped_labs = labs_table.groupby(['LabName', 'PatientID']).aggregate(np.mean)


In [146]:
grouped_labs

LabValue
PatientID                              LabName                                 
016A590E-D093-4667-A5DA-D68EA6987D93_1 CBC: ABSOLUTE LYMPHOCYTES      21.050000
                                       CBC: ABSOLUTE NEUTROPHILS      69.300000
                                       CBC: BASOPHILS                  0.125000
                                       CBC: EOSINOPHILS                0.275000
                                       CBC: HEMATOCRIT                43.500000
...                                                                         ...
FFCDECD6-4048-4DCB-B910-1218160005B3_3 METABOLIC: TOTAL PROTEIN        7.080000
                                       URINALYSIS: PH                  5.857143
                                       URINALYSIS: RED BLOOD CELLS     2.137500
                                       URINALYSIS: SPECIFIC GRAVITY    1.000000
                                       URINALYSIS: WHITE BLOOD CELLS   2.754545

[13003 rows x 1 columns]

In [160]:
# grouped_labs['PatientID'].tolist()

In [161]:
grouped_labs['LabValue'].keys()[0]

('016A590E-D093-4667-A5DA-D68EA6987D93_1', 'CBC: ABSOLUTE LYMPHOCYTES')

In [410]:
# labs_dict.keys() # checking if it returns in order

In [476]:
from collections import defaultdict
labs_dict = defaultdict(list)
labs_dict = {'pat 1':[1, 2, 3]}
labs_dict

{'pat 1': [1, 2, 3]}

In [477]:
labs_dict.keys()

dict_keys(['pat 1'])

In [478]:
for label, value in grouped_labs.iterrows():

    patient_id = label[0]
    
    if patient_id in labs_dict.keys():
        # print(patient_id + ' already exists')
        labs_dict[patient_id].append(value[0])
    else:
        
        labs_dict[patient_id] = []
        labs_dict[patient_id].append(value[0])


In [480]:
# labs_dict

In [470]:
len(labs_dict.keys())

373

# COMPARISON LOGIC

In [474]:
def Nmaxelements(list1, N):
    final_list = []
  
    for i in range(0, N): 
        max1 = 0
          
        for j in range(len(list1)):     
            if list1[j] > max1:
                max1 = list1[j];
                  
        list1.remove(max1);
        final_list.append(max1)
          
    print(final_list)


In [475]:
def compare_similar(new_patient_labs):

    sim_scores = []

    for patient in labs_dict:
                
        sim_score = 1 - np.mean(abs((labs_dict[patient] - new_patient_labs)))
        sim_scores.append(sim_score)
        
    Nmaxelements(sim_scores, 5)
        

# SETTING UP UI

In [ ]:
def show_instructions():
    
    popup('Welcome.', [
    put_html('Please download an Electronic Health Record Template for CBC, Metabolic, and Urinalysis Lab Results. Once you upload your patient\'s records, you will receive a diagnosis suggestion based on 5 patients with the most similar medical history.'),
        
    put_file(name='CBC Lab Results', content=b'Absolute Lymphocytes:\nAbsolute Neutrophil:\nBasophils:\nEosinophils:\nHematocrit:\nHemoglobin:\nLymphocytes:\nMCH:\nMCHC:\nMean Corpuscular Volume:\nMonocytes:\nNeutrophils:\nPlatelet Count:\nRDW:\nRed Blood Cell Count:\nWhite Blood Cell Count:', label='Download CBC Lab Results Template'),
    put_file(name='Metabolic Lab Results', content=b'Albumin:\nALK PHOS:\nALT/SGPT:\nANION GAP:\nAST/SGOT:\nBILI Total:\nBUN:\nCalcium\nCarbon Dioxide:\nChloride:\nCreatinine:\nGlucose:\nPotassium:\nSodium:\nTotal Protein:', label='Download Metabolic Lab Results Template'),
    put_file(name='Urinalysis Lab Results', content=b'pH:\nRed Blood Cells:\nSpecific Gravity:\nWhite Blood Cells:', label='Download Urinalysis Lab Results Template'),

    put_button('Got it!', onclick=close_popup)])

In [ ]:
def show_tests(btn_val):
    
    if btn_val == 'CBC':
        cbc_lab_results_file = file_upload("Patient's CBC Lab Results:", accept="txt/*")

    if btn_val == 'Metabolic':
        metabolic_lab_results_file = file_upload("Patient's Metabolic Lab Results:", accept="txt/*")

    if btn_val == 'Urinalysis':
        urinalysis_lab_results_file = file_upload("Patient's Urinalysis Lab Results:", accept="txt/*")

    
     # cbc_lab_results_file = file_upload("Patient's CBC Lab Results:", accept="txt/*")
    # metabolic_lab_results_file = file_upload("Patient's Metabolic Lab Results:", accept="txt/*")
    # urinalysis_lab_results_file = file_upload("Patient's Urinalysis Lab Results:", accept="txt/*")


In [ ]:
cbc_labs = []
metabolic_labs = []
urinalysis_lab = []

def read_labs(cbc_labs_fn, metabolic_labs_fn, urinalysis_labs_fn): 
    
    with open(cbc_labs_fn) as cbc_f:
        for line in cbc_f:
            print(line.split(':')[1]) # lab value
            string_val = line.split(':')[1]
            cbc_labs.append(float(string_val))
            
    with open(metabolic_labs_fn) as metabolic_f:
        for line in metabolic_f:
            print(line.split(':')[1]) # lab value
            string_val = line.split(':')[1]
            metabolic_labs.append(float(string_val))
            
    with open(urinalysis_labs_fn) as urinalysis_f:
        for line in urinalysis_f:
            print(line.split(':')[1]) # lab value
            string_val = line.split(':')[1]
            urinalysis_lab.append(float(string_val))
    

In [ ]:
read_labs('test results.txt', 'test results.txt', 'test results.txt')

In [ ]:
cbc_labs

In [ ]:
metabolic_labs

In [ ]:
urinalysis_lab

In [ ]:
def compare_similar(cbc_labs, metabolic_labs, urinalysis_lab):
    

In [ ]:
def analyze():
    print('analyzing')

In [ ]:
def welcome():
    
    # show_instructions()
    
    popup('Welcome.', [
    put_html('Please click on a lab test to enter specific results. Once you submit these values, you will receive a diagnosis suggestion based on 5 patients with the most similar medical history.'),
        
    put_file(name='CBC Lab Results', content=b'Absolute Lymphocytes:\nAbsolute Neutrophil:\nBasophils:\nEosinophils:\nHematocrit:\nHemoglobin:\nLymphocytes:\nMCH:\nMCHC:\nMean Corpuscular Volume:\nMonocytes:\nNeutrophils:\nPlatelet Count:\nRDW:\nRed Blood Cell Count:\nWhite Blood Cell Count:', label='Download CBC Lab Results Template'),
    put_file(name='Metabolic Lab Results', content=b'Albumin:\nALK PHOS:\nALT/SGPT:\nANION GAP:\nAST/SGOT:\nBILI Total:\nBUN:\nCalcium\nCarbon Dioxide:\nChloride:\nCreatinine:\nGlucose:\nPotassium:\nSodium:\nTotal Protein:', label='Download Metabolic Lab Results Template'),
    put_file(name='Urinalysis Lab Results', content=b'pH:\nRed Blood Cells:\nSpecific Gravity:\nWhite Blood Cells:', label='Download Urinalysis Lab Results Template'),

    put_button('Got it!', onclick=close_popup)])
    
    
    
    txt = put_text('Please click on a lab test to enter specific results. Once you submit these values, you will receive a diagnosis suggestion based on 5 patients with the most similar medical history.')    

    
    put_buttons(['CBC', 'Metabolic', 'Urinalysis'], onclick=show_tests)
    
    put_button('Begin Analysis', onclick=analyze)

In [ ]:
if __name__ == '__main__':
    welcome()